# Explanation of my solution

In this notebook, I am going through the thinking process I had to find a solution to this problem.

I will name this solution: **Moving the chairs to the room names**.

You can follow this code along in the `script.py` file at the root of the directory.



In [1]:


from utils_notebook import get_project_root, notebook_print_of

ROOT = get_project_root()
ROOMS_FOLDER = str(ROOT) + '/explanatory_rooms'

## Overall organization of the code

In the `script.py` file we have as:
- **Input**: a path to the `txt` file reprensenting an apartment. *ex: rooms.txt*
- **Output**: a print in the console of the excepted result, namely: 
```
- Number of different chair types for the apartment
- Number of different chair types per room
```
as mentioned in the **task_en.txt** file.

To perform this task, it runs the `run_solution()` function

The `run_solution()` is divided in 4 steps:

- Step 1: Read the `txt` file
- Step 2: Locate the chairs in the apartment
- Step 3: Find the room of each chair
- Step 4: Transform the result and display it

## Preliminary steps

### Step 1: Read and transform the txt file

I simply read the `room.txt` file or any other `txt` file of the same type and tranform it into a list of lists that I name `apartment_init`. 

I chose this *list of lists* data structure, i.e. to easily track any point in the apartment with coordinates: i, j as follows: `apartment_init[i][j]` which will be used in the rest of the code.

For debugging purposes, note that an apartment as a list of lists can be displayed in the console or the debugger with the function `console_print_of()` present in the `utils.py` file.

### Step 2: Locate chairs in the apartment

1. I scan throught every single pair of coordinates `(i, j)` to build the dictionary: `dict_pos_chairs` with:
    - **key**: (i, j) the coordinates of the chairs
    - **values**: the coresponding letter of chair in the apartment

2. In addition to this dictionary, I also create a list `list_pos_chairs` which gives the positions of chairs ordered vertically and horizontally. Sorting the list this way allows to easily find which chair is treated when we loop over each of them.



## Step 3: Find the room of each chair 
### (Main step)

This step is divided into 3 sub-steps:
- Step 3.1: Temporarily remove the other chairs in the apartment
- Step 3.2: Search the room of a chair
- Step 3.3: Save the result

This hole step is performed looping over the position of the chairs: `list_pos_chairs`. Within the loop the variables are:
- `k`: the index in `list_pos_chairs` of the chair. Having the `txt` file open, as `list_pos_chairs` is sorted it is fairly easy to locate the chair with `k`.
- `coord`: the coordinates of the chair in the `apartment`
-  `chair = apartment_init[i][j]`


I have already transformed the input string extracted from the txt file as list of list that name `apartment_init`. For each chair, when I explore the apartment and move the chair to find the room it is in, I temporarily remove the other chairs in the whole apartment to avoid any type of conflict.

This action is performed by the function: `remove_chairs()`. The result is saved as the list of lists: `apartment`.

This is how `apartment` can look like:

In [3]:
notebook_print_of(f'{ROOMS_FOLDER}/1.rooms_one_chair.txt')

['+-----------+------------------------------------+',
 '|           |                                    |',
 '| (closet)  |                                    |',
 '|           |                            S       |',
 '|           |         (sleeping room)            |',
 '|           |                                    |',
 '|           |                                    |',
 '+-----------+                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '|           |                                    |',
 '|           +--------------+---------------------+',
 '|                          |                     |',
 '|                          |                     |',
 '|                          |    (office)         |',
 '|                          |                     |',
 '+--------------+           |                     |',
 '|              |           |                     |',
 '| (toile

Once 